In [7]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
from PIL import Image
from sklearn.feature_extraction import image as skImage
from glob import glob
import numpy as np
import keras
import os

# -- DATA GETTER -- #
class DataGetter():
    """CLASS::DataGetter:"""
    def __init__(self, data_dir, img_size, classes, phase='train'):
        self.phase = phase
        self.data_dir = data_dir
        self.img_size = img_size
        self.classes = classes
        self.data_paths = []
        self.data = []
        self.label = []
    
    def load_data(self):
        for category in self.classes:
            self.data_paths.append(glob(self.data_dir+os.sep+self.phase+os.sep+category+os.sep+'*.jpg'))
        for k, category in enumerate(self.data_paths):
            for img_path in category:
                img = load_img(img_path)
                img = img.resize((self.img_size,self.img_size))
                self.data.append(np.expand_dims(img_to_array(img),axis=0))
                self.label.append(k)
    
    def __getitem__(self, key):
        if not isinstance(key, int):
            raise ValueError('Key has to be of type: int')
        return (self.data[key],self.label[key])
    
    def __len__(self):
        return len(self.data)

In [8]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Reshape, Activation
from keras import backend as K

# -- MODEL -- #
class Model_MLP():
    """CLASS::Model_MLP"""
    def __init__(self, config_dict, trained=False):
        model = self.get_model_structure(config_dict)
        if trained:
            model.load_weights(config_dict['weights_path'])
        layer_outputs = [layer.output for layer in model.layers]
        self.model = Model(inputs=model.inputs, outputs=layer_outputs)

    def get_model_structure(self,config_dict):
        model = Sequential()
        size = config_dict['img_size']
        model.add(Reshape((size*size*3,),input_shape=(size, size, 3)))
        for layer in config_dict['layers']:
            model.add(Dense(units=layer['units'], activation=layer['activation']))
        model.add(Dense(units=8, activation='softmax'))
        return model

    def predict(self,tensor):
        return self.model.predict(tensor)


In [9]:
pwd

'/content'

In [4]:
import zipfile
import os
with zipfile.ZipFile("MIT_split-20210124T132955Z-001.zip", 'r') as zip_ref:
    zip_ref.extractall(os.getcwd())

path1 = os.getcwd() + "/" + "MIT_split"
print(path1)

In [11]:
path1 = os.getcwd() + "/" + "MIT_split"
print(path1)

/content/MIT_split


# Experimentation With different Number of Layers

In [29]:
from keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
import os

import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt


# -- CONSTANTS -- #
EPOCHS = 100
PATCH_SIZE  = 64
PATCHES_DIR = path1
SAVEPATH = os.getcwd() + '/' + 'Week3'



#os.makedirs(SAVEPATH)

if __name__ == "__main__":
    if not os.path.isdir(SAVEPATH):
        os.mkdir(SAVEPATH)
    # -- DEFINE MODEL COMBINATIONS -- #
    model_comps = [
        {'layers':[{'units':2048,'activation':'relu'},{'units':1024,'activation':'relu'}],'img_size':PATCH_SIZE,'batch_size':16},
        {'layers':[{'units':3072,'activation':'relu'},{'units':768,'activation':'relu'}],'img_size':PATCH_SIZE,'batch_size':16},
        {'layers':[{'units':2048,'activation':'relu'}],'img_size':PATCH_SIZE,'batch_size':16},
        {'layers':[{'units':3072,'activation':'relu'}],'img_size':PATCH_SIZE,'batch_size':16},
        {'layers':[{'units':2048,'activation':'relu'},{'units':1024,'activation':'relu'}],'img_size':PATCH_SIZE,'batch_size':16},
        {'layers':[{'units':3072,'activation':'relu'},{'units':768,'activation':'relu'}],'img_size':PATCH_SIZE,'batch_size':16},
        {'layers':[{'units':2048,'activation':'relu'}],'img_size':PATCH_SIZE,'batch_size':16},
        {'layers':[{'units':3072,'activation':'relu'}],'img_size':PATCH_SIZE,'batch_size':16},
        {'layers':[{'units':2048,'activation':'relu'},{'units':1024,'activation':'relu'}],'img_size':PATCH_SIZE,'batch_size':16},
        {'layers':[{'units':3072,'activation':'relu'},{'units':768,'activation':'relu'}],'img_size':PATCH_SIZE,'batch_size':16},
        {'layers':[{'units':2048,'activation':'relu'}],'img_size':PATCH_SIZE,'batch_size':16},
        {'layers':[{'units':3072,'activation':'relu'}],'img_size':PATCH_SIZE,'batch_size':16},
    ]
    
    for ind, config_dict in enumerate(model_comps):
        # -- DEFINE CONSTANTS -- #
        MODEL_NAME = 'model'+str(ind)
        MODEL_FNAME = SAVEPATH+'/'+MODEL_NAME+'.h5'
        BATCH_SIZE = config_dict['batch_size']

        # -- CREATE MODEL -- #
        mlp_patches = Model_MLPatches(config_dict, phase='train', trained=False)
        mlp_patches.model.compile(
            loss='categorical_crossentropy',
            optimizer='sgd',
            metrics=['accuracy']
        )
        print(mlp_patches.model.summary())
        plot_model(mlp_patches.model, to_file=SAVEPATH+'/'+MODEL_NAME+'_structure.png', show_shapes=True, show_layer_names=True)

        print('Done\n')

        print('Loading Data...')
        train_datagen = ImageDataGenerator(
          rescale=1./255,
          horizontal_flip=True)
        test_datagen = ImageDataGenerator(rescale=1./255)

        print('train_generator')
        train_generator = train_datagen.flow_from_directory(
            PATCHES_DIR+'/train',  # this is the target directory
            target_size=(PATCH_SIZE, PATCH_SIZE),  # all images will be resized to PATCH_SIZExPATCH_SIZE
            batch_size=BATCH_SIZE,
            classes = ['coast','forest','highway','inside_city','mountain','Opencountry','street','tallbuilding'],
            class_mode='categorical')  # since we use binary_crossentropy loss, we need categorical labels

        print('validation_generator')
        validation_generator = test_datagen.flow_from_directory(
            PATCHES_DIR+'/test',
            target_size=(PATCH_SIZE, PATCH_SIZE),
            batch_size=BATCH_SIZE,
            classes = ['coast','forest','highway','inside_city','mountain','Opencountry','street','tallbuilding'],
            class_mode='categorical')
  
        history = mlp_patches.model.fit_generator(
            train_generator,
            steps_per_epoch=(1881) // BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=validation_generator,
            validation_steps=(807) // BATCH_SIZE)

        mlp_patches.model.save_weights(MODEL_FNAME)  # always save your weights after training or during training

        
        # summarize history for accuracy
        '''
        plt.plot(history.history['acc'])
        plt.plot(history.history['val_acc'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        plt.savefig(SAVEPATH+'/'+MODEL_NAME+'_accuracy.png')
        plt.close()
        # summarize history for loss
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        plt.savefig(SAVEPATH+'/'+MODEL_NAME+'_loss.png')
        plt.close()
        '''

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_32 (Reshape)         (None, 12288)             0         
_________________________________________________________________
dense0 (Dense)               (None, 2048)              25167872  
_________________________________________________________________
dense1 (Dense)               (None, 1024)              2098176   
_________________________________________________________________
dense_42 (Dense)             (None, 8)                 8200      
Total params: 27,274,248
Trainable params: 27,274,248
Non-trainable params: 0
_________________________________________________________________
None
Done

Loading Data...
train_generator
Found 1881 images belonging to 8 classes.
validation_generator
Found 807 images belonging to 8 classes.
Epoch 1/100


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


117/117 [==============================] - 5s 43ms/step - loss: 2.1556 - accuracy: 0.1940 - val_loss: 1.6383 - val_accuracy: 0.4175
Epoch 2/100
117/117 [==============================] - 5s 41ms/step - loss: 1.6489 - accuracy: 0.3872 - val_loss: 1.6296 - val_accuracy: 0.3487
Epoch 3/100
117/117 [==============================] - 5s 42ms/step - loss: 1.5067 - accuracy: 0.4642 - val_loss: 1.4454 - val_accuracy: 0.5150
Epoch 4/100
117/117 [==============================] - 5s 42ms/step - loss: 1.3488 - accuracy: 0.5252 - val_loss: 1.3820 - val_accuracy: 0.5300
Epoch 5/100
117/117 [==============================] - 5s 43ms/step - loss: 1.3357 - accuracy: 0.5186 - val_loss: 1.5190 - val_accuracy: 0.4288
Epoch 6/100
117/117 [==============================] - 5s 42ms/step - loss: 1.2762 - accuracy: 0.5488 - val_loss: 1.2735 - val_accuracy: 0.5625
Epoch 7/100
117/117 [==============================] - 5s 42ms/step - loss: 1.2313 - accuracy: 0.5671 - val_loss: 1.2939 - val_accuracy: 0.5450
Epoc

In [13]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Reshape
from keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os

# -- DEFINE CONSTANTS -- #
config_dict = {
    'layers':[{'units':2048,'activation':'relu'},{'units':1024,'activation':'relu'}],
    'img_size':64,
    'weights_path': os.getcwd() + '/model0.h5'
    }
classes = ['coast','forest','highway','inside_city','mountain','Opencountry','street','tallbuilding']
data_dir = path1
 
if __name__ == "__main__":

    print('Load Model')
    # -- LOAD Model -- #
    model = Model_MLP(config_dict)
    print(config_dict['weights_path'])
    print('Loaded')

    # -- LOAD DATA -- #
    print('Loading train data...')
    train_getter = DataGetter(data_dir, config_dict['img_size'], classes)
    train_getter.load_data()
    print('Loaded.')

    # -- PREDICT DATA -- #
    print('Predicting train data...')
    outputs = []
    labels = []
    for k,(tensor,label) in enumerate(train_getter):
        outputs.append(model.predict(tensor))
        labels.append(label)
        print('Image [{0}] Predicted.'.format(k))
    print('Done')

    with open('./task2_train_feat.pkl','wb') as file:
        pickle.dump(outputs,file)
    with open('./task2_train_label.pkl','wb') as file:
        pickle.dump(labels,file)
    
    # -- LOAD DATA -- #
    print('Loading test data...')
    test_getter = DataGetter(data_dir, config_dict['img_size'], classes, phase='test')
    test_getter.load_data()
    print('Loaded.')

    # -- PREDICT DATA -- #
    print('Predicting test data...')
    outputs = []
    labels = []
    for k,(tensor,label) in enumerate(test_getter):
        outputs.append(model.predict(tensor))
        labels.append(label)
        print('Image [{0}] Predicted.'.format(k))
    print('Done')

    with open('./task2_test_feat.pkl','wb') as file:
        pickle.dump(outputs,file)
    with open('./task2_test_label.pkl','wb') as file:
        pickle.dump(labels,file)

Load Model
/content/model0.h5
Loaded
Loading train data...
Loaded.
Predicting train data...
Image [0] Predicted.
Image [1] Predicted.
Image [2] Predicted.
Image [3] Predicted.
Image [4] Predicted.
Image [5] Predicted.
Image [6] Predicted.
Image [7] Predicted.
Image [8] Predicted.
Image [9] Predicted.
Image [10] Predicted.
Image [11] Predicted.
Image [12] Predicted.
Image [13] Predicted.
Image [14] Predicted.
Image [15] Predicted.
Image [16] Predicted.
Image [17] Predicted.
Image [18] Predicted.
Image [19] Predicted.
Image [20] Predicted.
Image [21] Predicted.
Image [22] Predicted.
Image [23] Predicted.
Image [24] Predicted.
Image [25] Predicted.
Image [26] Predicted.
Image [27] Predicted.
Image [28] Predicted.
Image [29] Predicted.
Image [30] Predicted.
Image [31] Predicted.
Image [32] Predicted.
Image [33] Predicted.
Image [34] Predicted.
Image [35] Predicted.
Image [36] Predicted.
Image [37] Predicted.
Image [38] Predicted.
Image [39] Predicted.
Image [40] Predicted.
Image [41] Predi

# Applying SVM

In [14]:
import numpy as np

def histogram_intersection_kernel(a, b):
    K = np.empty(shape=(a.shape[0], b.shape[0]), dtype=np.float32)
    for i in range(a.shape[0]):
        K[i] = np.sum(np.minimum(a[i], b), axis=1)
    return K

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [15]:
class Model_MLPatches():
    """CLASS::Model_MLPatches"""
    def __init__(self,config_dict, phase='train', trained=False):
        self.model = self.get_model_structure(config_dict, phase)
        if trained:
            self.model.load_weights(config_dict['weights_path'])

    def get_model_with_layer_name(self,layer_name):
        return Model(inputs=self.model.inputs,outputs=self.model.get_layer(layer_name).output)
        
    def get_model_structure(self,config_dict, phase):
        model = Sequential()
        size = config_dict['img_size']
        model.add(Reshape((size*size*3,),input_shape=(size, size, 3)))
        for k,layer in enumerate(config_dict['layers']):
            model.add(Dense(units=layer['units'], activation=layer['activation'],name='dense{0}'.format(k)))
        if phase is 'train':
            model.add(Dense(units=8, activation='softmax'))
        else:
            model.add(Dense(units=8, activation='linear'))
        return model

    def predict(self,tensor):
        return self.model.predict(tensor)

In [16]:
import numpy as np

from sklearn.cluster import MiniBatchKMeans


class VisualWords():

    def __init__(self, n_clusters):
        self.n_clusters = n_clusters

    def fit(self, descriptors):
        self.codebook = MiniBatchKMeans(n_clusters=self.n_clusters,
            verbose=False,
            batch_size=self.n_clusters*20,
            compute_labels=False,
            reassignment_ratio=10**-4,
            random_state=42)
        descriptors = np.vstack(descriptors)
        self.codebook.fit(descriptors)

    def get_visual_words(self, descriptors):
        visual_words = np.empty((len(descriptors), self.n_clusters), dtype=np.float32)
        for i, descriptor in enumerate(descriptors):
            words = self.codebook.predict(descriptor)
            visual_words[i,:] = np.bincount(words, minlength=self.n_clusters)
        return visual_words

class VisualWordsPyramid():

    def __init__(self, n_clusters):
        self.n_clusters = n_clusters

    def fit(self, descriptors):
        self.codebook = MiniBatchKMeans(n_clusters=self.n_clusters,
            verbose=False,
            batch_size=self.n_clusters*20,
            compute_labels=False,
            reassignment_ratio=10**-4,
            random_state=42)
        full_descriptors = []
        for des in descriptors:
            to_append = []
            for subdes in des:
                to_append.extend(subdes)
            full_descriptors.append(to_append)
        full_descriptors = np.vstack(full_descriptors)
        self.codebook.fit(full_descriptors)

    # def get_visual_words(self, descriptors):
    #     visual_words = np.empty((len(descriptors), self.n_clusters*len(descriptors[0])), dtype=np.float32)
    #     for i, descriptor in enumerate(descriptors):
    #         for j, subdescriptor in enumerate(descriptor):
    #             words = self.codebook.predict(subdescriptor)
    #             visual_words[i,j*self.n_clusters:(j+1)*self.n_clusters] = np.bincount(words, minlength=self.n_clusters)
    #     return visual_words

    def get_visual_words(self, descriptors):
        visual_words = np.empty((len(descriptors), self.n_clusters*(len(descriptors[0])+1)), dtype=np.float32)
        for i, descriptor in enumerate(descriptors):
            full_descriptor = []
            for subdes in descriptor:
                full_descriptor.extend(subdes)
            words = self.codebook.predict(full_descriptor)
            visual_words[i,:self.n_clusters] = np.bincount(words, minlength=self.n_clusters)

            for j, subdescriptor in enumerate(descriptor):
                words = self.codebook.predict(subdescriptor)
                visual_words[i,(j+1)*self.n_clusters:(j+2)*self.n_clusters] = np.bincount(words, minlength=self.n_clusters)
        return visual_words

In [31]:
import numpy as np
import pickle
import os

from PIL import Image
from sklearn.feature_extraction import image as skImage
# from utils.Kernels import histogram_intersection_kernel, softmax
# from utils.Model import Model_MLPatches
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
# from utils.VisualWords import VisualWords

from sklearn.cluster import MiniBatchKMeans

MODEL_DIR = os.getcwd() + '/Week3/model11.h5'
DATASET_DIR = path1
PATCH_SIZE = 64
LAYER = 'dense0'

if __name__ == "__main__":
    print('Loading Model..')
    config_dict = {
        'layers':[{'units':3072,'activation':'relu'}],
        'img_size':PATCH_SIZE,
        'weights_path':MODEL_DIR
    
    }
    mlp_model = Model_MLPatches(config_dict,phase='test',trained=True)
    feat_extractor = mlp_model.get_model_with_layer_name(LAYER)
    features = feat_extractor.output.shape[1]
    print('Loaded')

    train_dir = DATASET_DIR+'/train'
    test_dir = DATASET_DIR+'/test'
    classes = {'coast':0,'forest':1,'highway':2,'inside_city':3,'mountain':4,'Opencountry':5,'street':6,'tallbuilding':7}
    num_patches = int(256/PATCH_SIZE)**2

    if not os.path.isfile('Week3/task5_train_desc_2.pkl'):
        print('Get train descriptors...')
        num_img = 1881
        count=0
        train_desc = []
        for class_dir in os.listdir(train_dir):
            cls = classes[class_dir]
            for imname in os.listdir(os.path.join(train_dir,class_dir)):
                im = Image.open(os.path.join(train_dir,class_dir,imname))
                patches = skImage.extract_patches_2d(np.array(im), (PATCH_SIZE, PATCH_SIZE), max_patches=num_patches)
                out = feat_extractor.predict(patches/255.)
                train_desc.append(out)
                count += 1
        with open('Week3/task5_train_desc_2.pkl','wb') as file:
            pickle.dump(train_desc,file)

    if not os.path.isfile('Week3/task5_test_desc_2.pkl'):
        print('Get test descriptors...')
        num_img = 807
        count=0
        test_desc = []
        for class_dir in os.listdir(test_dir):
            cls = classes[class_dir]
            for imname in os.listdir(os.path.join(test_dir,class_dir)):
                im = Image.open(os.path.join(test_dir,class_dir,imname))
                patches = skImage.extract_patches_2d(np.array(im), (PATCH_SIZE, PATCH_SIZE), max_patches=num_patches)
                out = feat_extractor.predict(patches/255.)
                test_desc.append(out)
                count += 1
        with open('Week3/task5_test_desc_2.pkl','wb') as file:
            pickle.dump(test_desc,file)

    train_labels = pickle.load(open('task2_train_label.pkl','rb'))
    test_labels = pickle.load(open('task2_test_label.pkl','rb'))
    train_desc = pickle.load(open('Week3/task5_train_desc_2.pkl','rb'))
    test_desc = pickle.load(open('Week3/task5_test_desc_2.pkl','rb'))

    results_train = []
    results_test = []
    best_params = []
    N_CLUSTERS = [128, 256, 512, 1024]
    for k,n_cluster in enumerate(N_CLUSTERS):
        print('Train Visual Words...')
        visualWords = VisualWords(n_cluster)
        visualWords.fit(train_desc)
        feature_train = visualWords.get_visual_words(train_desc)
        feature_test = visualWords.get_visual_words(test_desc)
        print('Trained.')

        K_FOLDS = 5
        PARAM_GRID = {'C': [0.001389, 0.01, 0.1, 1], 'kernel': [histogram_intersection_kernel], 'gamma': [1e-3, 1e-4, 'scale']}

        cv = GridSearchCV(SVC(), param_grid=PARAM_GRID, cv=K_FOLDS, n_jobs=-1, verbose=5)
        cv.fit(feature_train, train_labels)

        results_train.append(cv.score(feature_train, train_labels))
        results_test.append(cv.score(feature_test, test_labels))
        best_params.append(cv.best_params_)

        print("Train accuracy score: {}\nTest accuracy score: {}\nBest params: {}\n".format(results_train[-1], results_test[-1], cv.best_params_))
        print("All results: {}".format(cv.cv_results_))
        print('CLUSTER: {0}'.format(n_cluster))
    
    with open('Week3/task5_results_2.pkl','wb') as file:
        pickle.dump((results_train,results_test,best_params),file)

Loading Model..
Loaded
Get train descriptors...
Get test descriptors...
Train Visual Words...
Trained.
Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   47.7s finished


Train accuracy score: 0.5390749601275917
Test accuracy score: 0.43866171003717475
Best params: {'C': 0.01, 'gamma': 0.001, 'kernel': <function histogram_intersection_kernel at 0x7f05c77fe598>}

All results: {'mean_fit_time': array([1.15183463, 1.1144331 , 1.21177363, 1.23597813, 1.22925515,
       1.22979169, 1.24300141, 1.25235453, 1.25770545, 1.36136646,
       1.34179606, 1.27591562]), 'std_fit_time': array([0.06858321, 0.07908577, 0.00909694, 0.0135922 , 0.02549342,
       0.01727217, 0.01387497, 0.00448138, 0.00296472, 0.01967411,
       0.04032944, 0.04528149]), 'mean_score_time': array([0.28782506, 0.2900423 , 0.30185204, 0.30612359, 0.30458121,
       0.30722237, 0.30362482, 0.3057126 , 0.30509562, 0.30063887,
       0.30162201, 0.2733098 ]), 'std_score_time': array([0.0166974 , 0.0107861 , 0.00539509, 0.00590133, 0.00358776,
       0.00621746, 0.00465614, 0.00147469, 0.00438852, 0.00314019,
       0.00399577, 0.04525204]), 'param_C': masked_array(data=[0.001389, 0.001389, 0.00

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.3min finished


Train accuracy score: 0.7602339181286549
Test accuracy score: 0.44981412639405205
Best params: {'C': 0.1, 'gamma': 0.001, 'kernel': <function histogram_intersection_kernel at 0x7f05c77fe598>}

All results: {'mean_fit_time': array([2.08523998, 2.06638379, 2.06535654, 2.05480647, 2.05985641,
       2.04840465, 2.07080035, 2.0777071 , 2.06592617, 2.1963521 ,
       2.15063729, 2.14543152]), 'std_fit_time': array([0.06531006, 0.03261763, 0.02785528, 0.0340289 , 0.01437351,
       0.04529686, 0.01193434, 0.03494619, 0.01061312, 0.0441279 ,
       0.02489398, 0.13502491]), 'mean_score_time': array([0.53717089, 0.54147282, 0.52714696, 0.53552046, 0.52357574,
       0.52024984, 0.51334291, 0.5225482 , 0.51477799, 0.52320275,
       0.51159248, 0.48450875]), 'std_score_time': array([0.02712358, 0.03760563, 0.01406005, 0.02699575, 0.0133173 ,
       0.01736063, 0.00606943, 0.01508437, 0.00834699, 0.01412681,
       0.00564836, 0.08848572]), 'param_C': masked_array(data=[0.001389, 0.001389, 0.001

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   38.2s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  2.7min finished


Train accuracy score: 0.8463583200425305
Test accuracy score: 0.43866171003717475
Best params: {'C': 0.1, 'gamma': 0.001, 'kernel': <function histogram_intersection_kernel at 0x7f05c77fe598>}

All results: {'mean_fit_time': array([4.12681623, 4.21534929, 4.19103642, 4.23379259, 4.2224493 ,
       4.18347054, 4.24361329, 4.16164055, 4.25517826, 4.25210123,
       4.23947959, 3.97125015]), 'std_fit_time': array([0.06539348, 0.08144531, 0.07264437, 0.08968844, 0.11044687,
       0.07557764, 0.08957614, 0.04384409, 0.07078512, 0.08370881,
       0.02563407, 0.58775521]), 'mean_score_time': array([1.04966121, 1.08546338, 1.05509753, 1.07935452, 1.06443276,
       1.06692753, 1.06222663, 1.07371883, 1.06581469, 1.06866288,
       1.07042446, 0.97823615]), 'std_score_time': array([0.0343994 , 0.06253325, 0.02966489, 0.04086204, 0.0316233 ,
       0.03312362, 0.05789181, 0.04073155, 0.03685017, 0.02723637,
       0.01984078, 0.15052909]), 'param_C': masked_array(data=[0.001389, 0.001389, 0.001

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.9min finished


Train accuracy score: 0.9138755980861244
Test accuracy score: 0.436183395291202
Best params: {'C': 0.1, 'gamma': 0.001, 'kernel': <function histogram_intersection_kernel at 0x7f05c77fe598>}

All results: {'mean_fit_time': array([7.83242245, 7.90555854, 7.90894108, 7.88882093, 7.84117126,
       7.89069939, 7.767208  , 7.86411381, 7.81504807, 7.88123913,
       7.81082263, 7.33760653]), 'std_fit_time': array([0.21256069, 0.06083288, 0.07030625, 0.05734805, 0.02149737,
       0.16421162, 0.21583331, 0.24817392, 0.26866658, 0.27890032,
       0.28257415, 1.3314244 ]), 'mean_score_time': array([1.98171024, 2.00207829, 2.00112529, 2.00625234, 1.97356801,
       1.96719122, 1.91705103, 1.93966889, 1.94544683, 1.9316586 ,
       1.90757642, 1.7779573 ]), 'std_score_time': array([0.05325597, 0.03212866, 0.01289112, 0.02676971, 0.02606403,
       0.09361342, 0.05819558, 0.0659612 , 0.0468448 , 0.04245098,
       0.04077482, 0.36643734]), 'param_C': masked_array(data=[0.001389, 0.001389, 0.00138

Here we are applying the SVM and we are getting a train accuracy of 91% and a test accuracy of 43% 